In [0]:
from pyspark.sql import functions as F
from delta.tables import *

TB_CHAMADO = "v_credit.silver.tb_chamado"
TB_LOG = "v_credit.silver.tb_chamado_log"
TB_CUSTO = "v_credit.silver.tb_custo_chamado"
TB_PESQUISA = "v_credit.silver.tb_pesquisa"

TABELA_DESTINO = "v_credit.gold.ft_atendimento_geral"

In [0]:
df_chamado = spark.read.table(TB_CHAMADO)
df_log = spark.read.table(TB_LOG)
df_custo = spark.read.table(TB_CUSTO)
df_pesquisa = spark.read.table(TB_PESQUISA)

df_join = (
    df_chamado.alias("c")
    .join(df_log.alias("l"), on="cd_chamado", how="left")
    .join(df_custo.alias("cust"), on="cd_chamado", how="left")
    .join(df_pesquisa.alias("p"), on="cd_chamado", how="left")
)

df_gold = df_join.select(
    F.md5(F.concat(
        F.coalesce(F.col("c.cd_cliente").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_motivo").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_canal").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_atendente").cast("string"), F.lit("0")),
        F.coalesce(F.col("c.cd_chamado").cast("string"), F.lit("0"))
    )).alias("pk_fato_atendimento"),
    
    F.col("c.cd_cliente").cast("string"),
    F.col("c.cd_motivo"),
    F.col("c.cd_canal"),
    F.col("c.cd_atendente"),
    
    F.col("c.cd_chamado").cast("string"),
    
    F.to_date(F.coalesce(F.col("l.dh_inicio"), F.col("c.dt_ingestion"))).alias("dt_referencia"),
    F.hour(F.coalesce(F.col("l.dh_inicio"), F.col("c.dt_ingestion"))).cast("smallint").alias("nu_hora_dia"),

    F.coalesce(F.col("c.tm_espera"), F.lit(0)).cast("bigint").alias("val_tempo_espera"),
    F.coalesce(F.col("c.tm_duracao"), F.lit(0)).cast("bigint").alias("val_tempo_atendimento"),
   
    F.coalesce(F.col("cust.vl_custo"), F.lit(0)).cast("decimal(12,10)").alias("val_custo"),
    
    F.col("p.nu_nota").cast("smallint").alias("val_nota_csat"),
    
    F.when(F.col("c.st_resolvido") == True, 1).otherwise(0).cast("smallint").alias("st_resolvido"),
    F.lit(1).cast("smallint").alias("qtd_interacoes"),
    
    F.when(
        (F.col("c.tm_espera") > 300) | (F.col("p.nu_nota") <= 2), 1
    ).otherwise(0).cast("smallint").alias("fl_experiencia_negativa"),
    
    F.current_timestamp().alias("dt_ingestion"),
    F.lit("ETL_Databricks_Merge").alias("dc_origem")
)

deltaTable = DeltaTable.forName(spark, TABELA_DESTINO)

deltaTable.alias("target").merge(
    df_gold.alias("source"),
    "target.pk_fato_atendimento = source.pk_fato_atendimento"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Merge realizado com sucesso em {TABELA_DESTINO}")